In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import matplotlib
matplotlib.use("Agg")
import numpy as np
from matplotlib import pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.pooling import AveragePooling2D
from keras.applications import ResNet50
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from keras.optimizers import SGD
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten, Lambda
from keras.layers.convolutional import Conv3D, MaxPooling3D, ZeroPadding3D
from keras.layers.convolutional import Convolution3D
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import pickle
import cv2
import os
import random
from tqdm import tqdm

path_test = '/content/drive/My Drive/Testing_set/Testing_set'
path_train = '/content/drive/My Drive/Training_set/Training'


In [ ]:
def get_image(input):
  ret = []
  vs = cv2.VideoCapture(input)
  
  while True:
    (grabbed, frame) = vs.read()
    if not grabbed:
      break    
    ret.append(frame)

  vs.release()
  cv2.destroyAllWindows()
  return ret
# 12900 minimum category length
def data_set():
  if os.path.exists('/content/train'): return
  os.makedirs('train', exist_ok = True)
  for folder in os.listdir(path_train):
    print(folder)
    imgs = []
    for video in tqdm(os.listdir(os.path.join(path_train, folder))):
      video_path = os.path.join(os.path.join(path_train, folder ),  video)
      imgs += get_image(video_path)
    random.shuffle(imgs)

    os.makedirs(os.path.join('/content/train', folder))
    for i in tqdm(range(0, 9000)):
      cv2.imwrite('/content/train/' + folder + '/' + str(i) + '.jpg'  , imgs[i])

    print(len(imgs))

data_set()

In [ ]:


mean = np.array([123.68, 116.779, 103.939], dtype="float32")
print("[INFO] loading images...")
LABELS = {'Diving': 0, 'Jumping': 1, 'Basketball': 2, 'Tennis': 3, 'Walking': 4}
data = []
labels = []
new_path = '/content/train'
for folder in os.listdir(new_path):
	cnt = 0
	for name in tqdm(os.listdir(os.path.join(new_path,folder))):
		if cnt >= 4000:break
		cnt += 1
		img = cv2.imread(os.path.join(os.path.join(new_path,folder), name))
		img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
		img = cv2.resize(img, (171, 128)).astype("float32")
		img-=mean
		data.append(img)
		labels.append(LABELS[folder])
		#imagePath = Diving + '/' + imagePath

print(len(data))



In [ ]:
data = np.array(data)
labels = np.array(labels)
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [ ]:
trainX = []
testX = []
trainY = []
testY = []
print(len(data))
print(len(labels))

In [ ]:
#lb = LabelBinarizer()
#labels = lb.fit_transform(labels)
#(trainX, testX, trainY, testY) = train_test_split(data, labels,test_size=0.0, stratify=labels, random_state=42)
#trainX = data
#trainY = labels
(trainX, testX, trainY, testY) = train_test_split(data, labels,test_size=.25, stratify=labels, random_state=42)


In [ ]:
print(len(trainX))
print(len(trainY))
print(len(testX))
print(len(testY))
data = []
labels = []

In [ ]:
data = None
trainAug = ImageDataGenerator(
	rotation_range=30,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

valAug = ImageDataGenerator()

mean = np.array([123.68, 116.779, 103.939], dtype="float32")
trainAug.mean = mean
valAug.mean = mean

In [ ]:
def c3d(summary=False):
    """ Return the Keras model of the network
    """
    model = Sequential()
    # 1st layer group
    model.add(Convolution3D(64, 3, 3, 3, activation='relu', 
                            border_mode='same', name='conv1',
                            subsample=(1, 1, 1), 
                            input_shape=(16, 112, 112, 3)))
    model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), 
                           border_mode='valid', name='pool1'))
    # 2nd layer group
    model.add(Convolution3D(128, 3, 3, 3, activation='relu', 
                            border_mode='same', name='conv2',
                            subsample=(1, 1, 1)))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), 
                           border_mode='valid', name='pool2'))
    # 3rd layer group
    model.add(Convolution3D(256, 3, 3, 3, activation='relu', 
                            border_mode='same', name='conv3a',
                            subsample=(1, 1, 1)))
    model.add(Convolution3D(256, 3, 3, 3, activation='relu', 
                            border_mode='same', name='conv3b',
                            subsample=(1, 1, 1)))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), 
                           border_mode='valid', name='pool3'))
    # 4th layer group
    model.add(Convolution3D(512, 3, 3, 3, activation='relu', 
                            border_mode='same', name='conv4a',
                            subsample=(1, 1, 1)))
    model.add(Convolution3D(512, 3, 3, 3, activation='relu', 
                            border_mode='same', name='conv4b',
                            subsample=(1, 1, 1)))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), 
                           border_mode='valid', name='pool4'))
    # 5th layer group
    model.add(Convolution3D(512, 3, 3, 3, activation='relu', 
                            border_mode='same', name='conv5a',
                            subsample=(1, 1, 1)))
    model.add(Convolution3D(512, 3, 3, 3, activation='relu', 
                            border_mode='same', name='conv5b',
                            subsample=(1, 1, 1)))
    model.add(ZeroPadding3D(padding=(0, 1, 1)))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), 
                           border_mode='valid', name='pool5'))
    model.add(Flatten())
    # FC layers group
    model.add(Dense(4096, activation='relu', name='fc6'))
    model.add(Dropout(.5))
    model.add(Dense(4096, activation='relu', name='fc7'))
    model.add(Dropout(.5))
    model.add(Dense(487, activation='softmax', name='fc8'))
    if summary:
        print(model.summary())
    return model


In [ ]:
base = c3d()
base = Model(inputs=base.input,
                      outputs=base.get_layer('fc7').output)
head = base.output
head = Dense(5, activation="softmax")(head)
model = Model(inputs=base.input, outputs=head)

In [ ]:
model.summary()

In [ ]:
print("[INFO] compiling model...")
opt = SGD(lr=1e-4, momentum=0.9, decay=1e-4 / 25)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])
# train the head of the network for a few epochs (all other layers
# are frozen) -- this will allow the new FC layers to start to become
# initialized with actual "learned" values versus pure random
print("[INFO] training head...")


In [ ]:
from keras.callbacks import ModelCheckpoint
filepath="weights2.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
# Fit the model
#model.fit(X, Y, validation_split=0.33, epochs=25, batch_size=64, callbacks=callbacks_list)

In [ ]:
#H = model.fit_generator(trainAug.flow(trainX, trainY, batch_size=32),	validation_data=valAug.flow(testX, testY),	epochs=10)

H = model.fit(trainX, trainY, epochs=5, 
          validation_data=(testX, testY), 
					shuffle=1, batch_size=64, callbacks=callbacks_list)
'''model.fit(
	trainAug.flow(trainX, trainY, batch_size=32),
	steps_per_epoch=len(trainX) // 32,
	validation_data=valAug.flow(testX, testY),
	validation_steps=len(testX) // 32,
	epochs=15)'''

In [ ]:
#finalmodel = Model(inputs=baseModel.input, outputs=headModel)
#finalmodel.load_weights("weights.best.hdf5")
#finalmodel.compile(loss="categorical_crossentropy", optimizer=opt,	metrics=["accuracy"])
model.load_weights("weights.best.hdf5")
#model.save('vision1.h5')

In [ ]:


# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=32)
predictions = np.asarray(predictions)
testY = np.asarray(testY)
'''print(classification_report(testY.argmax(axis=1),
      predictions.argmax(axis=1),target_names=lb.classes_))'''

'''print(testY.argmax(axis=1))
print(predictions.argmax(axis=1))
print(lb.classes_)'''
N = 25
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig('plot.png')



In [ ]:
ok = {}
#LABELS = {'Diving': 0, 'Jumping': 1, 'Basketball': 2, 'Tennis': 3, 'Walking': 4}

for i in range(1, 127):
  st = 'test_image(' + str(i) + ').mpg'
  if i < 32:
    ok[st]=0
  elif i < 52:
    ok[st]=1
  elif i < 87:
    ok[st]=2
  elif i < 106:
    ok[st]=3
  else:
    ok[st]=4

In [ ]:
from pandas import DataFrame as pd
import csv
file = []
t = 0
for folder in os.listdir(path_test):
  vid_path = os.path.join(path_test, folder)
  imgs = []
  
  #ret = []
  vs = cv2.VideoCapture(vid_path)
  
  while True:
    (grabbed, frame) = vs.read()
    if not grabbed:
      break    
    
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (224, 224)).astype("float32")
    frame-=mean
    imgs.append(frame)

  vs.release()
  cv2.destroyAllWindows()  
  
  y = model.predict(np.array(imgs))
  lst = 5*[0.0]
  lst = np.array(lst)
  for i in y:
    lst += i
    #lst[np.argmax(i)]+=1
  
  idx = np.argmax(np.array(lst))
  file.append([folder, idx])
  folder = folder.replace(" ", "")
  #print(idx, folder)
  t += int(int(idx)==int(ok[folder]))

pd(np.array(file), columns=['Id', 'Category']).to_csv('submit.csv', index = False)

In [ ]:
H = None

In [ ]:
print(t/126*100.0)